# Ejemplo de EDA: Análisis Exploratorio de Datos

EDA es la sigla en inglés para Exploratory Data Analysis y consiste en una de las primeras tareas que
tiene que desempeñar el Científico de Datos. Es cuando revisamos por primera vez los datos que nos
llegan, por ejemplo un archivo CSV y deberemos intentar comprender “¿de qué se trata?”, vislumbrar
posibles patrones y reconocer distribuciones estadísticas que puedan ser útiles en el futuro.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm  #!pip install statsmodels

## Carga de archivo csv desde una URL

In [ ]:
url = 'https://raw.githubusercontent.com/lorey/list-of-countries/master/csv/countries.csv'
df = pd.read_csv(url, sep=";") #index_col=0
display(df.head(5))
print(df.head(5))

In [ ]:
# ver las primeras filas 
df.head()

In [ ]:
# ver las ultimas  filas 
df.tail()

In [ ]:
# ver las exemplos de filas 
df.sample()

In [ ]:
# guardamos en dataframe crudo en un archivo csv
# df.to_csv("lista_de_paises.csv") 

## Conocer información básica

In [ ]:
print('Cantidad de Filas y columnas:',df.shape)
print('Nombre columnas:',end="\n\t\t")
print(*df.columns,sep=", ")

In [ ]:
df.info()

Pandas filtra las features numéricas y calcula datos estadísticos que pueden ser útiles: cantidad,
media, desvío estándar, valores máximo y mínimo.

In [ ]:
df.describe()

## Matriz de Correlación

Verifiquemos si hay correlación entre los datos

In [ ]:
corr = df.corr()
display(corr)
sm.graphics.plot_corr(corr, xnames=list(corr.columns))
plt.show()

En este caso vemos baja correlación entre las variables. Dependiendo del algoritmo que utilicemos
podría ser una buena decisión eliminar features que tuvieran alta correlación
Cargamos un segundo archivo csv para ahondar en el crecimiento de la población en los últimos
años, filtramos a España y visualizamos

## Cargar una segunda fuente de datos

In [ ]:
url = 'https://raw.githubusercontent.com/DrueStaples/Population_Growth/master/countries.csv'
df_pop = pd.read_csv(url)
print(df_pop.head(5))

In [ ]:
df_pop.sample(10)

In [ ]:
df_pop.to_csv("Poblacion_crecimiento.csv")

### Aqui vemos la población año tras año de España

In [ ]:
df_pop_es = df_pop[df_pop["country"] == 'Spain' ]
df_pop_es.head()

In [ ]:
df_pop_es.drop([1416],axis=0)

In [ ]:
df_pop_es.shape

## Visualicemos datos

In [ ]:
df_pop_es.drop(['country'],axis=1)['population'].plot(kind='bar')

In [ ]:
df_pop_es.drop(['country'],axis=1).set_index("year").plot()

In [ ]:
df_pop_ar = df_pop[(df_pop["country"] == 'Argentina')]
df_pop_ar.head()

In [ ]:
df_pop_ar.shape

In [ ]:
df_pop_ar.set_index('year').plot(kind='bar')

## Comparativa entre 2 países

In [ ]:
anios  = df_pop_es['year'].unique()
pop_ar = df_pop_ar['population'].values
pop_es = df_pop_es['population'].values

df_plot = pd.DataFrame({'Argentina': pop_ar,
                    'Spain': pop_es}, 
                       index=anios)
df_plot.plot(kind='bar')

In [ ]:
df_plot.plot()

In [ ]:
plt.plot(anios, pop_ar  ,'go-', label = "argentina")
plt.plot(anios, pop_es  ,'bo-', label = "españa")
plt.legend()
plt.show()

# Filtremos paises hispano-hablantes

In [ ]:
df_espanol = df.replace(np.nan, '', regex=True)
df_espanol = df_espanol[ df_espanol['languages'].str.contains('es') ]
df_espanol

In [ ]:
df_espanol.shape

## Visualicemos por población

In [ ]:
df_espanol.set_index('alpha_3')[['population','area']].plot(kind='bar',rot=65,figsize=(20,10))

## Detección de Outliers

In [ ]:
anomalies = []

# Funcion ejemplo para detección de outliers
def find_anomalies(data):
    # Establecer límite superior e inferior a 2 desviación estándar
    data_std  = data.std()
    data_mean = data.mean()
    anomaly_cut_off = data_std * 2
    lower_limit     = data_mean - anomaly_cut_off 
    upper_limit     = data_mean + anomaly_cut_off
    print(lower_limit.iloc[0])
    print(upper_limit.iloc[0])

    # Generate outliers
    for index, row in data.iterrows():
        outlier = row # # obtener primer columna
        # print(row)
        if (outlier.iloc[0] > upper_limit.iloc[0]) or \
           (outlier.iloc[0] < lower_limit.iloc[0]):
            anomalies.append(index)
    return anomalies

anomalies = find_anomalies(df_espanol[['population']])

In [ ]:
anomalies

In [ ]:
# Quitemos BRA y USA por ser outlies y volvamos a graficar:
df_espanol.drop(anomalies, inplace = True)

In [ ]:
df_espanol.set_index('alpha_3')[['population','area']].plot(kind='bar',rot=65,figsize=(20,10))

## Graficamos ordenando por tamaño Población

In [ ]:
df_espanol.set_index('alpha_3')[['population','area']].sort_values(["population"]).plot(kind='bar',rot=65,figsize=(20,10))

## Visualización por Área

In [ ]:
df_espanol.set_index('alpha_3')[['area']].sort_values(["area"]).plot(kind='bar',rot=65,figsize=(20,10))

In [ ]:
# En este caso, podriamos quitar por "lo bajo", area menor a 110.000 km2:
df_2 = df_espanol.set_index('alpha_3')
df_2 = df_2[df_2['area'] > 110000]
df_2

In [ ]:
df_2[['area']].sort_values(["area"]).plot(kind='bar',rot=65,figsize=(20,10))

In [ ]:
df_espanol.set_index('alpha_3')[['population','area']].plot(kind='bar',rot=0,figsize=(20,15),width=0.8)

In [ ]:
df_espanol.set_index('alpha_3')[
    ['population','area']].sort_values(
    ["population"]).plot(kind='bar',rot=65,figsize=(20,10),width=0.8)

In [ ]:

df_2[['area']].sort_values(["area"]).plot(kind='pie',subplots=True,rot=65,figsize=(20,10))

In [ ]:
df_2.area.plot(kind = "pie",)